# Ispezione del dataset e controlli di qualità

Questo notebook è dedicato all’ispezione preliminare del dataset e ai controlli di qualità dei dati.
L’obiettivo è comprendere la struttura generale delle feature e delle variabili target, verificare l’assenza di anomalie (valori mancanti, duplicati o disallineamenti) e garantire la coerenza dei dati prima di procedere con l’analisi esplorativa statistica e la modellazione.

In [35]:
from src.dataset import load_dataset

### Caricamento del dataset
In questa sezione vengono caricati i dataset di training contenenti le feature operative (X) e le variabili target (y).
L’obiettivo è verificare che i dati siano accessibili e correttamente separati prima di procedere con qualsiasi analisi.


In [37]:
df_x= load_dataset("data/train/X_train.csv")
df_y = load_dataset("data/train/y_train.csv")

print("Dimensioni dataset X:", df_x.shape)
print("Dimensioni dataset y:", df_y.shape)

X shape: (742625, 8)
y shape: (742625, 3)


### Struttura generale del dataset
Viene analizzata la struttura generale dei dataset osservando:
1) le prime righe;
2) i nomi delle colonne;
3) il tipo di variabili presenti.
Questo consente di comprendere il contenuto informativo delle feature e delle variabili target.

In [39]:
print("Colonne di X:")
print(df_x.columns.tolist())

print("\nColonne di y:")
print(df_y.columns.tolist())

display(df_x.head())
display(df_y.head())


Colonne di X:
['id', 'trq_measured', 'oat', 'mgt', 'pa', 'ias', 'np', 'ng']

Colonne di y:
['id', 'faulty', 'trq_margin']


,id,trq_measured,oat,mgt,pa,ias,np,ng
0,0,54.100,2.00000,544.5000,212.1408,74.56250,89.18000,99.6400
1,1,49.625,24.22231,578.4844,1625.6400,30.35596,99.55273,91.3866
2,2,52.000,7.00000,566.1000,1912.9250,65.62500,100.14000,90.9600
3,3,62.400,7.25000,560.1000,277.0632,54.81250,90.64000,100.2800
4,4,62.900,23.25000,593.7000,53.6448,73.43750,99.91000,92.1700


,id,faulty,trq_margin
0,0,1,-13.717745
1,1,0,1.791863
2,2,1,-13.944871
3,3,0,-0.017281
4,4,0,7.322404


### Tipologia delle variabili e informazioni generali
In questa fase si analizzano i tipi di dato, il numero di osservazioni e l’eventuale presenza di valori mancanti.
Questo controllo è fondamentale per prevenire errori nelle successive fasi di preprocessing.

In [40]:
print("Informazioni su X:")
df_x.info()

print("\nInformazioni su y:")
df_y.info()

Informazioni su X:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742625 entries, 0 to 742624
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            742625 non-null  int64  
 1   trq_measured  742625 non-null  float64
 2   oat           742625 non-null  float64
 3   mgt           742625 non-null  float64
 4   pa            742625 non-null  float64
 5   ias           742625 non-null  float64
 6   np            742625 non-null  float64
 7   ng            742625 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 45.3 MB

Informazioni su y:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742625 entries, 0 to 742624
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          742625 non-null  int64  
 1   faulty      742625 non-null  int64  
 2   trq_margin  742625 non-null  float64
dtypes: float64(1), int64(2)
memory usag

#### Verifica della presenza di valori mancanti (NaN)
Si controlla la presenza di valori mancanti all’interno delle feature e delle variabili target.
I valori mancanti possono compromettere sia l’analisi esplorativa sia l’addestramento dei modelli, pertanto è necessario individuarli in questa fase preliminare.


In [43]:
nan_x_count = df_x.isna().sum()
nan_y_count = df_y.isna().sum()

if nan_x_count.sum() == 0 and nan_y_count.sum() == 0:
    print(" Nessun valore mancante rilevato in X e y.")
else:
    print("Attenzione: sono presenti valori mancanti nel dataset.")


 Nessun valore mancante rilevato in X e y.


#### Individuazione di osservazioni duplicate
In questa sezione si verifica la presenza di righe duplicate nei dataset.
Le osservazioni duplicate possono introdurre bias e alterare le analisi successive, pertanto è necessario individuarle ed eventualmente rimuoverle.

In [44]:
print("Numero di righe duplicate in X:", df_x.duplicated().sum())
print("Numero di righe duplicate in y:", df_y.duplicated().sum())

Numero di righe duplicate in X: 0
Numero di righe duplicate in y: 0
